# Исследование набора данных

## Загрузка датасета

In [1]:
import kagglehub

c:\Programs\Python\Neural-tech\Lab3\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = kagglehub.dataset_download("carlosrunner/pizza-not-pizza")
print(path)

100%|██████████| 101M/101M [00:04<00:00, 23.7MB/s] 

Extracting files...


C:\Users\FPostaushkin\.cache\kagglehub\datasets\carlosrunner\pizza-not-pizza\versions\1


## Исследование данных

In [4]:
from PIL import Image
import os

In [13]:
not_pizza = os.path.join(path, "pizza_not_pizza")
not_pizza = os.path.join(not_pizza, "not_pizza")

not_pizza_images = os.listdir(not_pizza)
not_pizza_images = [os.path.join(not_pizza, image) for image in not_pizza_images if image.endswith(".jpg") or image.endswith(".png")]

widths = []
heights = []
for image in not_pizza_images:
    try:
        with Image.open(image) as img:
            width, height = img.size
            widths.append(width)
            heights.append(height)
    except Exception as e:
        print(f"Ошибка при обработке файла {image}: {e}")
        
print(f"Кол. не пицц - {len(not_pizza_images)}, ширина и высота min{(min(widths), min(heights))}, max{(max(widths), max(heights))}")

Кол. не пицц - 983, ширина и высота min(288, 287), max(512, 512)


In [14]:
pizza = os.path.join(path, "pizza_not_pizza")
pizza = os.path.join(pizza, "pizza")

pizza_images = os.listdir(pizza)
pizza_images = [os.path.join(pizza, image) for image in pizza_images if image.endswith(".jpg") or image.endswith(".png")]

widths = []
heights = []
for image in pizza_images:
    try:
        with Image.open(image) as img:
            width, height = img.size
            widths.append(width)
            heights.append(height)
    except Exception as e:
        print(f"Ошибка при обработке файла {image}: {e}")
        
print(f"Кол. не пицц - {len(pizza_images)}, ширина и высота min{(min(widths), min(heights))}, max{(max(widths), max(heights))}")

Кол. не пицц - 983, ширина и высота min(287, 287), max(512, 512)


## Аугментация и расширение датасета

In [12]:
import albumentations as A
import numpy as np

In [27]:
class AlbumentationsAugmentation:
    def __init__(self):
        self.transform = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.Rotate(limit=15, p=0.5),
            A.RandomBrightnessContrast(p=0.5)
        ])
        
    def __call__(self, img):
        img = np.array(img)
        augmented = self.transform(image=img)
        return augmented['image']
    
augmenter = AlbumentationsAugmentation()

In [16]:
path_to_save = "./data"

In [31]:
for i, image in enumerate(not_pizza_images):
    try:
        with Image.open(image) as img:
            img.save(path_to_save + f"/not_pizza_{i}_{0}.jpg")

            for j in range(4):
                augmented_img = augmenter(img)
                augmented_img = Image.fromarray(augmented_img)
                augmented_img.save(path_to_save + f"/not_pizza_{i}_{j + 1}.jpg")

    except Exception as e:
        print(f"Ошибка при обработке файла {image}: {e}")

In [32]:
for i, image in enumerate(pizza_images):
    try:
        with Image.open(image) as img:
            img.save(path_to_save + f"/pizza_{i}.jpg")

            for j in range(4):
                augmented_img = augmenter(img)
                augmented_img = Image.fromarray(augmented_img)
                augmented_img.save(path_to_save + f"/pizza_{i}_{j + 1}.jpg")
    except Exception as e:
        print(f"Ошибка при обработке файла {image}: {e}")

In [5]:
from model import PizzaCNN

model = PizzaCNN(version=1)
print(model)

PizzaCNN(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=16384, out_features=64, bias=True)
    (8): ReLU()
    (9): Linear(in_features=64, out_features=1, bias=True)
  )
)
